In [27]:
import random
import numpy as np

from deap import base, creator, tools, algorithms
from scipy.stats import bernoulli

from tensorflow.keras.datasets import fashion_mnist

In [111]:
import init_model

In [110]:
import importlib
importlib.reload(init_model)

<module 'init_model' from '/Users/longnguyen/Desktop/School/SEM6/CS410.O21_MNVTGDT/Project/genCNNv2/init_model.py'>

In [36]:
def get_bit_idx(num_nodes):
    L =  0 # genome length
    BITS_INDICES = np.empty((0, 2),dtype = np.int32)
    start = 0
    end = 0

    for x in num_nodes:
        end = end + sum(range(x))
        BITS_INDICES = np.vstack([BITS_INDICES, [start, end]])
        start = end
    L = end

    return L, BITS_INDICES

In [5]:
# Load the dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Normalize the images to a range of 0 to 1
train_images, test_images = train_images / 255.0, test_images / 255.0

# Reshape the data to include the channel dimension
train_images = train_images.reshape((train_images.shape[0], 28, 28, 1))
test_images = test_images.reshape((test_images.shape[0], 28, 28, 1))


In [126]:
STAGES = np.array(["s1", "s2"]) # S
NUM_NODES = np.array([3, 4]) # K
NUM_LABELS = 10
FILTERS = np.array([32, 64])
L, BITS_INDICES = get_bit_idx(NUM_NODES)

POPULATION_SIZE = 20
NUM_GEN = 20

TRAINING_EPOCHS = 10

In [84]:
def evaluate_model(individual):
    model = init_model.generate_model(individual, STAGES, NUM_NODES, BITS_INDICES, FILTERS, NUM_LABELS)

    model.fit(train_images, train_labels, epochs=TRAINING_EPOCHS, validation_data=(test_images, test_labels))
    
    test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
    
    return test_acc, 


In [127]:
random.seed(11)

# Kiểm tra creator đã được khởi tạo với hai lớp FitnessMax và Individual hay chưa
if not hasattr(creator, "FitnessMax"):
    # Tạo ra một lớp FitnessMax mở rộng từ lớp Fitness (base class của DEAP),
    # được sử dụng để tối đa hoá đơn mục tiêu, giá trị Fitness càng cao càng tốt (weights=(1.0,))
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
if not hasattr(creator, "Individual"):
    # Tạo ra một lớp Individual có dạng là list, thuộc tính fitness là một instance của FitnessMax
    creator.create("Individual", list, fitness=creator.FitnessMax)


toolbox = base.Toolbox()
hof = tools.HallOfFame(POPULATION_SIZE * L)

# Đăng ký một hàm có tên là binary được dùng để tạo ra một giá trị
#nhị phân theo phân phối bernoulli với xác suất 0.5 cho 1 và 0.5 cho 0
toolbox.register("binary", bernoulli.rvs, 0.5) 

# Đăng lý một hàm có tên là individual được gọi lặp lại để tạo ra một cá thể có L biến nhị phân,
# sử dụng lớp Individual được tạo ở trên 
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.binary, n = L)

# Đăng ký một hàm có tên là population được gọi lặp lại để tạo ra một quần thể các cá thể (list individual), 
# mỗi cá thể được tạo ra bằng hàm individual
toolbox.register("population", tools.initRepeat, list , toolbox.individual)

# Đăng ký một hàm lai ghép có tên là mate sử dụng hàm cxOrdered để hoán vị các biến
toolbox.register("mate", tools.cxOrdered)

# Đăng ký một hàm đột biến tên là mutate, sử dụng hàm mutShuffleIndexes để hoán đổi các index của các biến
# với xác suất đột biến là 0.8
toolbox.register("mutate", tools.mutShuffleIndexes, indpb = 0.8)

# Đăng ký một hàm chọn lọc với tên là select, sử dụng chiến lược chọn lọc roulette wheel
toolbox.register("select", tools.selRoulette)

# Đăng ký một hàm đánh giá tên là evaluate sử dụng hàm evaluateModel được định nghĩa ở trên
toolbox.register("evaluate", evaluate_model)

# Tạo ra một population với số lượng các cá thể là n
popl = toolbox.population(n = POPULATION_SIZE)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("min", np.min)
stats.register("max", np.max)

# Sử dụng simple evolutionary algorithm được cung cấp với DEAP
# với popl được tạo ra ở trên
# xác suất lai ghép là 0.4
# xác suất đột biến là 0.05
# số lượng thế hệ
# và in ra thông tin của quá trình thực thi

pop, log = algorithms.eaSimple(popl, toolbox, cxpb = 0.4, mutpb = 0.05, ngen = NUM_GEN, stats=stats, halloffame=hof, verbose = True)

Epoch 1/10


In [107]:
# print top optimal solutions 
best_individuals = tools.selBest(popl, k = 3)
for bi in best_individuals:
    print(bi, " ", bi.fitness.values[0])

[0, 0, 0]   0.8374000191688538
[0, 0, 0]   0.8267999887466431


In [ ]:
model = init_model.generate_model(best_individuals[0], STAGES, NUM_NODES, BITS_INDICES, FILTERS, NUM_LABELS)
model.summary()